In [ ]:
import numpy as np
import skccm as ccm
import pysindy as ps
import matplotlib.pyplot as plt
import os
from mpl_toolkits.mplot3d import Axes3D

from scipy.integrate import solve_ivp
from skccm.utilities import train_test_split
from pysindy.optimizers import SR3,SSR,FROLS,TrappingSR3
from pysindy.feature_library import CustomLibrary,PolynomialLibrary,FourierLibrary,GeneralizedLibrary

In [ ]:
def lv2(t, x, alpha=0.1, beta=0.02, delta=0.02, gamma=0.4):
    return [
        alpha * x[0] - beta * x[0] * x[1],
        delta * x[0] * x[1] - gamma * x[1],
    ]

def lv2_sys(x,y,time,dt,n): 
    integrator_keywords = {}
    integrator_keywords['rtol'] = 1e-12
    integrator_keywords['method'] = 'RK45'
    integrator_keywords['atol'] = 1e-12
    t_train = np.arange(0, time, dt)
    
    x1_train = [x, y]
    t_train_span = (t_train[0], t_train[-1])  
    lv_2 = solve_ivp(lv2, t_train_span, x1_train, t_eval=t_train, **integrator_keywords).y.T     
    plt.figure(n)
    plt.plot(lv_2[:,0])
    plt.plot(lv_2[:,1])

    return lv_2


def sys_sin(rx2, f, start, ts_length,n):
   
    x1 = np.sin(0.5*np.pi+np.linspace(0,(f-0.5)*np.pi,ts_length))*5  + 30
    x2 = np.zeros(ts_length)
   
    


    plt.figure(n)
    plt.plot(x1)   

    return x1,x2

def ccm_sys(lag,embed,x_1,x_2,n,name='None'):
    e1 = ccm.Embed(x_1)
    e2 = ccm.Embed(x_2)
    x1 = e1.embed_vectors_1d(lag,embed)
    x2 = e2.embed_vectors_1d(lag,embed)
    
    x1tr, x1te, y1tr, y1te = train_test_split(x1,x2, percent=.60)
    CCM_all = ccm.CCM()
    len_tr = len(x1tr)
    lib_lens = np.arange(10, len_tr, 10, dtype='int')
    CCM_all.fit(x1tr,y1tr)    
    x1p, x2p = CCM_all.predict(x1te, y1te,lib_lengths=lib_lens)   
    sc_xy_1,sc_xy_2 = CCM_all.score() 
    plt.figure(n)
    plt.plot(sc_xy_1,label='sc1')
    plt.plot(sc_xy_2,label='sc2')
    return sc_xy_1,sc_xy_2


functions = [ lambda x: x,
              lambda x,y : x*y]

lib_Fourier = FourierLibrary()
lib_Polynomial= PolynomialLibrary()
lib_custom = CustomLibrary(library_functions=functions)
lib_generalized = GeneralizedLibrary([lib_Polynomial, lib_Fourier])



opt = SSR(alpha=.2)

fig = plt.figure()  

In [ ]:
import csv
lv2_1 = lv2_sys(30, 3, 1000, 1, 9)

sys_sin_1,sys_sin_2 = sys_sin(3.8, 90.5,0.2, 1000, 88)

plt.figure(222)



plt.plot(sys_sin_1[:1000],label='inteference')
#plt.plot(sys_sin_2)
plt.plot(lv2_1[:1000,0],label='prey')
plt.plot(lv2_1[:1000,1],label='predator')
plt.legend()
plt.xlabel("Time t", size = 16,)
plt.ylabel("Population", size = 16,)

LV12_1,LV12_2=ccm_sys(1,3,lv2_1[:,0],lv2_1[:,1],21,name='LV12')

sinLV1_1,sinLV1_2=ccm_sys(1,3,sys_sin_1,lv2_1[:,0],22,name='sin+LV1')

sinLV2_1,sinLV2_2=ccm_sys(1,3,sys_sin_1,lv2_1[:,1],23,name='sin+LV2')

In [ ]:
sys_1 = np.stack((sys_sin_1, lv2_1[:,0], lv2_1[:,1]), axis=1)
model1 = ps.SINDy(optimizer= opt,
                    feature_library=lib_Polynomial

                  )
                
model1.fit(sys_1, t=1)
model1.print()
score1 = model1.score(sys_1,t=1)
print(score1)

In [ ]:
from scipy import interpolate
list_old=np.linspace(0,1000,1000)
list_new=np.linspace(0,1000,100000)
f_linear_1=interpolate.interp1d(list_old,lv2_1[:,0])
f_linear_2=interpolate.interp1d(list_old,lv2_1[:,1])
f_linear_3=interpolate.interp1d(list_old,sys_sin_1)
result_y_1=f_linear_1(list_new)
result_y_2=f_linear_2(list_new)
result_sin=f_linear_3(list_new)
sys_1 = np.stack((result_y_1, result_y_2), axis=1)

model5 = ps.SINDy(optimizer= opt,
                    feature_library=lib_Polynomial
                  )


model5.fit(sys_1, t=0.01)
model5.print()
